In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
import re
import nltk
from nltk.util import pr
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words("english"))

In [52]:
data=pd.read_csv(r"C:\Users\KIIT\Downloads\twitter_data.csv")

In [53]:
data.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [54]:
data.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [55]:
data.dtypes

Unnamed: 0             int64
count                  int64
hate_speech            int64
offensive_language     int64
neither                int64
class                  int64
tweet                 object
dtype: object

In [56]:
data.shape

(24783, 7)

In [57]:
data['labels']=data['class'].map({0:"Hate speech detected",1:"Offensive language detected",3:"No hate and offensive speech"})

In [58]:
data.head(10)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,NaN
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive language detected
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive language detected
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive language detected
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive language detected
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",Offensive language detected
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",Offensive language detected
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,Offensive language detected
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",Offensive language detected
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",Offensive language detected


In [59]:
data=data[['tweet','labels']]

In [60]:
data.head()

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,NaN
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive language detected
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive language detected
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive language detected
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive language detected


In [61]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
     # Split text into words
    text = [word for word in text.split() if word not in stopword]
    # Join the words back into a string
    text = " ".join(text)
    # Stem the words
    text = [stemmer.stem(word) for word in text.split()]
    # Join the stemmed words back into a string
    text = " ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean) 
print(data.head())

                                               tweet  \
0  rt mayasolov woman shouldnt complain clean hou...   
1   rt boy dat coldtyga dwn bad cuffin dat hoe place   
2  rt urkindofbrand dawg rt ever fuck bitch start...   
3             rt cganderson vivabas look like tranni   
4  rt shenikarobert shit hear might true might fa...   

                        labels  
0                          NaN  
1  Offensive language detected  
2  Offensive language detected  
3  Offensive language detected  
4  Offensive language detected  


In [62]:

data['tweet'] = data['tweet'].fillna('')
data['labels'] = data['labels'].fillna('')

# Convert to numpy arrays
x = np.array(data["tweet"])
y = np.array(data["labels"])

# Vectorize the text data
cv = CountVectorizer()
x = cv.fit_transform(x)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Initialize and train the classifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)



DecisionTreeClassifier()

In [66]:
test = "kill yourself"
data=cv.transform([test]).toarray()
print(clf.predict(data))

['Hate speech detected']


In [ ]:
#test = " you are bad i dont like you"
#data=cv.transform([test]).toarray()
#print(clf.predict(data))

#['Offensive language detected']